In [1]:
import urllib.request
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime

from collections import namedtuple
import pickle
import pandas as pd

from IPython.display import clear_output
from selenium import webdriver
from time import sleep
import json
from math import radians, cos, sin, asin, sqrt
import copy

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import time

In [2]:
atm = pickle.load(open('atms_target.dat','rb'))

In [3]:
atm.head()

,acqbin,caid,terminal,cardacceptorname,traget,log_traget
488,418388,ATM 0007,S1AM0007,SOVETSKAYA 47A SARANSK,46290000.0,17.650437
489,418388,ATM 0023,S1AM0023,SUVOROVA 44 IVANOVO,233770000.0,19.269848
490,418388,ATM 0024,S1AM0024,NEVSKOGO 7 KURSK,252395000.0,19.346506
491,418388,ATM 0042,S1AM0042,KIROVA 9A KALUGA,230980000.0,19.257842
492,418388,ATM 0070,S1AM0070,RIDZINSKOGO 24 YAKUTSK,290810000.0,19.488181


In [4]:
atm.columns  = ['atm_acqbin','atm_caid','atm_terminal','full_address','traget', 'log_traget']

In [5]:
atm['address'] = atm.full_address.apply(lambda x: x[:23].strip())

In [6]:
atm['city'] = atm.full_address.apply(lambda x: x[23:].strip()).apply(lambda x: x[:-2] if x.endswith(' G') else x)

In [7]:
def transform(x):
    dictionary = {"MOSKVA": "MOSCOW",
                  "SANKT-PETERBU": "SAINT PETERSBURG",
                  'S.-PETERBURG':  "SAINT PETERSBURG",
                  'ST-PETERSBURG': "SAINT PETERSBURG",
                  'ST PETERSBURG': "SAINT PETERSBURG"}
    for key, value in dictionary.items():
        x = x.replace(key, value)
    return x

In [8]:
atm.city = atm.city.map(transform)

In [9]:
atm['address_rus'] = ''
atm['lat'] = np.nan
atm['long'] = np.nan

In [10]:
to_replace = {
     'AV.': ' PROSPECT ',
     'AVE': ' PROSPECT ', 
     'PR-KT': ' PROSPECT ',
     'PR-T': ' PROSPECT ',
     'STR.': ' ULITSA ',
     ' ST ': ' ULITSA ',
     'UL.' : ' ULITSA ',
     'H/W.': ' SHOSSE ',
     ', LIT. ': '',
     'D.': '',
     'VLAD. ':'',
     ', KORP. ': '/',
     ' DZERZHINSK':  ' DZERZHINSKOGO ',
     '.': '. ',
     ',': ', ',
}

In [11]:
for i in atm.index:
    for key, value in to_replace.items():
        atm.at[i,'address'] =  atm.at[i,'address'].replace(key, value)

In [19]:
for i in atm.index:
    atm.at[i,'address'] = atm.at[i,'address'][4:] if atm.at[i,'address'][:4] == 'ABB ' else atm.at[i,'address']

# Парсер открытой информации из HeadHanter

In [12]:
def log_progress(sequence, every=10):
    for index, item in enumerate(sequence):
        if index % every == 0:
            clear_output()
            print(index)
        yield item

In [13]:
def parsing(atm):
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
    for i, atmi in log_progress(atm.iterrows()):
        urli = url + ' '.join([atmi.address, atmi.city]).replace(' ', '%20')
        driver.get(urli)
        try: 
            element = driver.find_element_by_xpath("//div[@class='clipboard__action-wrapper _inline']")
            atm.at[i, 'lat'],  atm.at[i, 'long'] = list(map(float, element.text.split(',')))

            addr_part1 = driver.find_element_by_xpath("//div[@class='card-title-view__title']")
            addr_part2 = driver.find_element_by_xpath("//div[@class='toponym-card-view__description']")
            atm.at[i, 'adress'] = addr_part1.text + ', ' + addr_part2.text 
        except:
            pass
        time.sleep(0.5)

In [24]:
pickle.dump(atm, open('atm_target.data','wb'))

In [22]:
parsing(atm)

0


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 6.1.7601 SP1 x86_64)
